In [2]:
import os
os.chdir('/Users/lucblassel')

In [3]:
import pandas as pd 
from Bio import SeqIO 
from Bio.Data import CodonTable
from itertools import product

In [4]:
datadir = "Documents/Work/hiv-drm-detection/data_pre_split/HIVDB_data/"

In [55]:
genetic_code = CodonTable.unambiguous_dna_by_name.get("Standard").forward_table

In [155]:
muts = {
    135: ["I", "L",],
    203: ["E", "K",],
    208: ["H", "Y",],
    218: ["D", "E",],
    228: ["L", "R", "H",],
}

In [156]:
muts_tuples = {
    135: [("I", "L"),],
    203: [("E", "K"),],
    208: [("H", "Y"),],
    218: [("D", "E"),],
    228: [("L", "R"), ("L", "H"),],
}

In [56]:
def translate(codon):
    return (codon.upper(), genetic_code.get(codon.upper(), (lambda x: "-" if x == "---" else "X")(codon)))

In [96]:
def merge_dicts(dicts):
    d = {}
    for dict_ in dicts:
        d = {**d, **dict_}
    return d

In [152]:
def hamming(s1, s2):
    return sum(
        c1 != c2
        for c1, c2 in 
        zip(s1.upper(), s2.upper())
    )

def get_weighted_distance(counts, start_aa, end_aa):
    crossed = product(
        counts.loc[start_aa].iteritems(),
        counts.loc[end_aa].iteritems()
    )

    return sum(
        hamming(s1, s2) * w1 * w2
        for (s1, w1), (s2, w2) in crossed
    )

# Checking codons in new mutations in UK sequences

In [57]:
alignmentCodons = {
    record.id: [
        (i + 1 , *translate(str(record[j:j+3].seq).upper()))
        for i, j in enumerate(range(0, len(record), 3))
        ]
    for record in SeqIO.parse(os.path.join(datadir, "UK", "total_RT.fa"), format="fasta")
}

In [106]:
interest = pd.DataFrame({
    k:merge_dicts([dict(**{f"{pos} codon": codon}, 
         **{f"{pos} AA": aa})
    for pos, codon, aa in codons
    if pos in muts])
    for k,codons in alignmentCodons.items()
}).transpose()


In [125]:
codonCounts = {}
for pos, aas in muts.items():
    codonCounts[pos] = (
        interest[interest[f"{pos} AA"].isin(aas)]
            .groupby(f"{pos} AA")
            [f"{pos} codon"]
            .apply(lambda x: x.value_counts(normalize=True))
    )

In [158]:
dists = {}
for pos, tuples in muts_tuples.items():
    for start, end in tuples:
        dists[f"{start}{pos}{end}"] = get_weighted_distance(codonCounts[pos], start, end)

In [160]:
{k:f"{v:.2f}" for k,v in dists.items()}

{'I135L': '1.16',
 'E203K': '1.31',
 'H208Y': '1.10',
 'D218E': '1.00',
 'L228R': '1.16',
 'L228H': '1.12'}

# Redo on African sequences

In [161]:
alignmentCodonsAfrica = {
    record.id: [
        (i + 1 , *translate(str(record[j:j+3].seq).upper()))
        for i, j in enumerate(range(0, len(record), 3))
        ]
    for record in SeqIO.parse(os.path.join(datadir, "Africa", "total_gaps.fa"), format="fasta")
}

In [162]:
interestAfrica = pd.DataFrame({
    k:merge_dicts([dict(**{f"{pos} codon": codon}, 
         **{f"{pos} AA": aa})
    for pos, codon, aa in codons
    if pos in muts])
    for k,codons in alignmentCodonsAfrica.items()
}).transpose()

In [163]:
codonCountsAfrica = {}
for pos, aas in muts.items():
    codonCountsAfrica[pos] = (
        interestAfrica[interestAfrica[f"{pos} AA"].isin(aas)]
            .groupby(f"{pos} AA")
            [f"{pos} codon"]
            .apply(lambda x: x.value_counts(normalize=True))
    )

In [164]:
distsAfrica = {}
for pos, tuples in muts_tuples.items():
    for start, end in tuples:
        distsAfrica[f"{start}{pos}{end}"] = get_weighted_distance(codonCountsAfrica[pos], start, end)

In [165]:
{k:f"{v:.2f}" for k,v in distsAfrica.items()}

{'I135L': '1.13',
 'E203K': '1.33',
 'H208Y': '1.12',
 'D218E': '1.00',
 'L228R': '1.21',
 'L228H': '1.17'}